#TensorFlow Tutorial with RNN

본 문서는 TensorFlow 를 사용하여 기초적인 RNN을 구현하고 실험하기 위한 실습 자료이다.

The code and comments are written by Dong-Hyun Kwak (imcomking@gmail.com)

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.


### Recurrent Neural Networks
Recurrent Neural Networks, 이하 RNN는 다음과 같은 구조를 가진 모델이다. RNN은 히든레이어에서 자기자신을 향하는 weight를 이용해 데이터간의 시간관계를 학습할 수 있다. 이러한 문제들을 시계열 학습이라고 부르며, 기존에 널리 쓰이던 Hidden Markov Model을 뉴럴넷을 이용해 구현했다고 볼 수 있다.


![](images/rnn.png)


![](images/unroll_rnn.png)

위의 구조는 1개의 Recurrent weight를 가진 hidden node이다. 이러한 hidden node들이 여러개를 모여 1개의 RNN layer를 형성하고, 이것이 다시 deep 하게 쌓이는 모델 또한 가능하다.(그러나 RNN은 deep 하게 쌓을 경우 학습이 쉽지 않다.)

RNN의 경우 MLP나 CNN에 비해서 구현이 다소 복잡하다. 따라서 RNN은 TensorFlow에서 제공하는 추상화된 API 들을 이용해서 구현하는 것이 일반적이다.

![](images/longterm.png)
(출처: http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

- tf.contrib.rnn.BasicRNNCell : 기본 RNN cell이외에 매우 다양한 cell을 사용할 수 있다.
https://www.tensorflow.org/api_guides/python/contrib.rnn#Core_RNN_Cells_for_use_with_TensorFlow_s_core_RNN_methods

- tf.contrib.seq2seq.sequence_loss : N개의 sequence의 cross-entropy를 weighted sum 한 Loss. weight는 0or1의 masking용도로써 쓰인다.
https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/sequence_loss

### Simple toy seq2seq with RNN / LSTM
<img src="images/rnn_seq2seq.jpg">
(이미지 출처: http://smile2x.tistory.com/archive/201605)
(소스코드 출처: https://github.com/hunkim/DeepLearningZeroToAll)

In [5]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(777)  # reproducibility

idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0 
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

y_data = [[1, 0, 2, 3, 3, 4]]    # ihello
y_one_hot = [[[0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 0, 1]]]  # o 4

input_dim = 5  # one-hot size
hidden_size = 10  # output from the LSTM. 5 to directly predict one-hot
output_dim = 5
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6

X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])  # X one-hot
Y = tf.placeholder(tf.float32, [None, sequence_length, output_dim])  # Y one-hot

cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
#cell = tf.contrib.rnn.GRUCell(num_units=hidden_size)
#cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)
print "outputs: ",outputs

outputs_flat = tf.reshape(outputs, [-1 , hidden_size])
print "outputs_flat: ",outputs_flat
w = tf.Variable(tf.random_normal([hidden_size, output_dim]))
b = tf.Variable(tf.random_normal([output_dim]))
y_prediction = tf.nn.softmax(tf.matmul(outputs_flat, w) + b)
print "y_prediction:", y_prediction
y_prediction = tf.reshape(y_prediction, [-1 , sequence_length, output_dim])
print "y_prediction:", y_prediction
print Y

y_label_pred = tf.argmax(y_prediction, 2)
    
loss = -tf.reduce_sum(Y*tf.log(y_prediction), name = 'cross_entropy')

train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_one_hot})
        if i %100 ==0:
            _y_prediction, _y_label_pred = sess.run([y_prediction, y_label_pred], feed_dict={X: x_one_hot})
            print i, "loss:", l, "prediction: ", _y_label_pred, "true Y: ", y_data

            # print char using dic
            result_str = [idx2char[c] for c in np.squeeze(_y_label_pred)]
            print "Prediction str: ", ''.join(result_str)

outputs:  Tensor("rnn/transpose:0", shape=(1, 6, 10), dtype=float32)
outputs_flat:  Tensor("Reshape:0", shape=(6, 10), dtype=float32)
y_prediction: Tensor("Softmax:0", shape=(6, 5), dtype=float32)
y_prediction: Tensor("Reshape_1:0", shape=(1, 6, 5), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 6, 5), dtype=float32)
0 loss: 13.3657 prediction:  [[1 0 1 1 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
Prediction str:  ihiilo
100 loss: 0.000930944 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
Prediction str:  ihello
200 loss: 0.000502015 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
Prediction str:  ihello
300 loss: 0.000329147 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
Prediction str:  ihello
400 loss: 0.000234132 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
Prediction str:  ihello
500 loss: 0.000175122 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
Prediction str:  ihello
600 loss: 0.00013602 prediction:  [[1 0 2 3 3 4]] tru

### Web Navigation Sequence Classification
앞서 살펴본 단순한 toy 문제에서 한 층 나아가, 52739개의 웹페이지를 탐색한 데이터를 학습한다. 빠른 학습을 위해 전체 데이터 중에서 아주 일부(100만개중 1000개만 가져옴)만을 가져왔기 때문에 test accuracy는 극히 낮으므로, train accuracy이 높아지는 것만 확인하도록 하자.

앞서 살펴본 예제는 seq2seq 이라는 형태의 RNN task이고, 본 예제는 전체 sequence를 보고서 어떤 class인지 예측하는 Sequence Classification 문제에 해당한다. 따라서 loss를 계산할 때 쓰이는 outputs가 어떻게 바뀌었는 지를 이해하는 것이 본 예제의 핵심이다.(마지막 step의 outputs만 가져옴)

- tf.one_hot : 자동으로 one_hot encoding을 해주는 함수
<br>https://www.tensorflow.org/api_docs/python/tf/one_hot


In [4]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
tf.reset_default_graph()

train_x = np.loadtxt('1000_index_x.txt')
train_y = np.loadtxt('1000_index_y.txt')

nclasses = 52739 #len(items)
nsteps = 4

dimhidden = 100
dimoutput = nclasses

x = tf.placeholder("int32", [None, nsteps])
y = tf.placeholder("int32", [None])
x_one_hot = tf.one_hot(x, nclasses, on_value=1.0, off_value=0.0 , dtype='float')
y_one_hot = tf.one_hot(y, nclasses, on_value=1.0, off_value=0.0, dtype='float')

cell = tf.contrib.rnn.BasicLSTMCell(dimhidden)
outputs, states = tf.nn.dynamic_rnn(cell, x_one_hot,  dtype=tf.float32)
last_output = outputs[:, -1, :]

w = tf.Variable(tf.random_normal([dimhidden, dimoutput]))
b = tf.Variable(tf.random_normal([dimoutput]))
prediction = tf.nn.softmax(tf.matmul(last_output, w) + b)

cost = -tf.reduce_mean( y_one_hot*tf.log(prediction+0.00000001))
optm     = tf.train.AdamOptimizer().minimize(cost) 
accr     = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(prediction,1), tf.argmax(y_one_hot,1)), tf.float32))

sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True)))
sess.run(tf.global_variables_initializer())

epochs = 1000
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

for i in range(epochs):
    _, _cost, _accr = sess.run([optm, cost, accr], feed_dict={x: x_train, y: y_train})
    if i%10 == 0 :
        print i, " : ", _cost , _accr
    
    if i%50 == 0 :
        _cost, test_acc = sess.run([cost, accr], feed_dict={x: x_test, y: y_test})
        print "Test accuracy: %.3f" % (test_acc)


0  :  0.000217181 0.0
Test accuracy: 0.000
10  :  0.000216318 0.0
20  :  0.000215133 0.0
30  :  0.000213787 0.0
40  :  0.000212321 0.0
50  :  0.000210751 0.0
Test accuracy: 0.000
60  :  0.000209084 0.0
70  :  0.000207321 0.0
80  :  0.000205456 0.0
90  :  0.000203478 0.0
100  :  0.000201371 0.0
Test accuracy: 0.000
110  :  0.000199111 0.0
120  :  0.000196676 0.00125
130  :  0.000194038 0.00125
140  :  0.000191169 0.0025
150  :  0.000188032 0.0025
Test accuracy: 0.000
160  :  0.000184587 0.00625
170  :  0.000180781 0.0175
180  :  0.000176544 0.03125
190  :  0.000171788 0.045
200  :  0.000166384 0.0625
Test accuracy: 0.000
210  :  0.000160157 0.10875
220  :  0.000152853 0.16
230  :  0.000144084 0.21
240  :  0.000133197 0.2875
250  :  0.000119178 0.38375
Test accuracy: 0.010
260  :  0.000100746 0.50375
270  :  7.69867e-05 0.6375
280  :  4.94459e-05 0.7575
290  :  2.52457e-05 0.86375
300  :  1.06855e-05 0.95125
Test accuracy: 0.025
310  :  4.18085e-06 0.98375
320  :  1.78268e-06 0.99875
330

## * Open-source TensorFlow Implementation

아래 링크는 TensorFlow로 구현되어 공개된 여러 오픈소스 프로젝트들을 모아서 정리해 둔 페이지들이다. 이중 본인의 연구 분야와 관련있는 프로젝트를 clone, 수정하여 사용할 경우 개발시간을 크게 단축할 수 있다.

https://github.com/tensorflow/models : Syntax Net, Magenta, Image2Txt
<br>https://github.com/TensorFlowKR/awesome_tensorflow_implementations
<br>https://github.com/aikorea/awesome-rl

유명한 오픈소스 몇가지를 살펴보자.